In [29]:
from tkinter import *
import tkinter.messagebox
from tkcalendar import DateEntry
import pyodbc
from datetime import date
import pandas as pd

#pega a data do dia
data = date.today()  


# conectar no banco de dados
dados_conexao = ("Driver={SQL Server};"
                 "Server=DESKTOP-GTUD9K4;"
                 "Database=ControleEstoque")
conexao = pyodbc.connect(dados_conexao)
cursor = conexao.cursor()


# procurar insumo -> ler o banco de dados
def btn_clicked0():
    entry0.delete("1.0","end")
    nome = entry1.get()
    #mostra os produtos que tem em estoque
    if nome == "":
        comando = f"""SELECT nome_insumo from Insumos"""
        cursor.execute(comando)
        for row in cursor.fetchall():
                    #mostrando no textbox
                    texto = f"{row.nome_insumo}\n"
                    entry0.insert('1.0', texto)
    else:
        comando = f"""SELECT count(*) FROM Insumos
                    where nome_insumo = '{nome}'"""
        cursor.execute(comando)

        # verificando se existe o insumo no banco de dados
        for row in cursor.fetchall():
            if row[0] != 0:
                comando2 = f"""SELECT * from Insumos
                WHERE nome_insumo = '{nome}';
               """
                cursor.execute(comando2)
                for row in cursor.fetchall():
                    #mostrando no textbox
                    texto = f"Item: {row.nome_insumo}\nValidade: {row.data_validade}\nLote:{row.lote}\nQuantidade:{row.qtde}\n"
                    entry0.insert('1.0', texto)
            else:
                tkinter.messagebox.showinfo(title='Aviso', message='Insumo não encontrado no sistema')
        
 

# deletar insumo -> deletar informação do banco de dados
def btn_clicked1():
    nome = entry1.get()
    operacao = 'deletar'
    qtde = float(entry4.get())
    comando = f"""SELECT count(*) FROM Insumos
                where nome_insumo = '{nome}'"""
    cursor.execute(comando)
    
    # verificando se existe o insumo no banco de dados
    for row in cursor.fetchall():
        if row[0] != 0:
            comando2 = f"""DELETE from Insumos
                    WHERE nome_insumo = '{nome}';
                    """
            cursor.execute(comando2)
            cursor.commit()
            
            # adicionando a operação realizada na tabela de relatório
            comando_operacoes = f"""INSERT INTO Operacoes(data, operacao, insumo, qtde)
            VALUES('{data}', '{operacao}', '{nome}', '{qtde}')"""
            cursor.execute(comando_operacoes)
            cursor.commit()
            
            tkinter.messagebox.showinfo(title='Retirada do sistema', message='Insumo deletado com sucesso')
        else:
            tkinter.messagebox.showinfo(title='Aviso', message='Insumo não encontrado no sistema')


# registrar uso -> atualizar o banco de dados
def btn_clicked2():
    nome = entry1.get()
    qtde = float(entry4.get())
    operacao = 'atualizar'
    comando0 = f"""SELECT count(*) FROM Insumos
                where nome_insumo = '{nome}'"""
    cursor.execute(comando0)
    
    # verificando se existe o insumo no banco de dados
    for row in cursor.fetchall():
        if row[0] != 0:
            comando = f"""SELECT qtde FROM Insumos
                        where nome_insumo = '{nome}'"""

            cursor.execute(comando)
            for row in cursor.fetchall():
                if row[0] - qtde > 0:
                    comando2 = f"""UPDATE Insumos
                        SET qtde = qtde - {qtde}
                        WHERE nome_insumo = '{nome}';
                        """
                    cursor.execute(comando2)
                    cursor.commit()
                    
                    # adicionando a operação realizada na tabela de relatório
                    comando_operacoes = f"""INSERT INTO Operacoes(data, operacao, insumo, qtde)
                    VALUES('{data}', '{operacao}', '{nome}', '{qtde}')"""
                    cursor.execute(comando_operacoes)
                    cursor.commit()
                    
                    tkinter.messagebox.showinfo(title='Aviso Update', message='Insumo atualizado com sucesso')
                
                # verificando se o insumo vai zerar com a redução de quantidades
                if row[0] - qtde == 0:
                    comando3 = f"""DELETE from Insumos
                            WHERE nome_insumo = '{nome}';
                            """
                    cursor.execute(comando3)
                    cursor.commit()
                    
                    # adicionando a operação realizada na tabela de relatório
                    comando_operacoes = f"""INSERT INTO Operacoes(data, operacao, insumo, qtde)
                    VALUES('{data}', '{operacao}', '{nome}', '{qtde}')"""
                    cursor.execute(comando_operacoes)
                    cursor.commit()
                    
                    tkinter.messagebox.showinfo(title='Aviso', message=f'O insumo {nome} foi 100% consumido e zerou todo no estoque, favor providenciar mais')
        else:
            tkinter.messagebox.showinfo(title='Aviso', message='Insumo não encontrado no sistema')
            
        
# adicionar insumo -> adicionar um novo insumo no banco de dados
def btn_clicked3():
    entry0.delete("1.0","end")
    nome = entry1.get()
    validade = entry2.get()
    lote = entry3.get()
    qtde = entry4.get()
    operacao = 'adicionar'
    comando0 = f"""SELECT count(*) FROM Insumos
                where nome_insumo = '{nome}'"""
    cursor.execute(comando0)
    
    # verificando se existe o insumo no banco de dados
    for row in cursor.fetchall():
        if row[0] == 0:
            comando = f"""INSERT INTO Insumos(nome_insumo, data_validade, lote, qtde)
            VALUES
                ('{nome}', '{validade}', '{lote}', '{qtde}')"""
            cursor.execute(comando)
            cursor.commit()
            
            # adicionando a operação realizada na tabela de relatório
            comando_operacoes = f"""INSERT INTO Operacoes(data, operacao, insumo, qtde)
            VALUES('{data}', '{operacao}', '{nome}', '{qtde}')"""
            cursor.execute(comando_operacoes)
            cursor.commit()
            
            tkinter.messagebox.showinfo(title='Cadastro', message=f'O insumo {nome} foi cadastrado com sucesso')
        else:
            
            #se já existir, ele vai ler a informação e mostrar no textbox
            comando2 = f"""SELECT * from Insumos
            WHERE nome_insumo = '{nome}';
           """
            cursor.execute(comando2)
            for row in cursor.fetchall():
                texto = f"Item: {row.nome_insumo}\nQuantidade: {row.data_validade}\nLote:{row.lote}\nValidade:{row.qtde}\n"
                entry0.insert('1.0', texto)
            tkinter.messagebox.showinfo(title='Aviso', message=f'O insumo {nome} já existe no sistema')
            
        
#exporta a tabela de operações para um excel
def gerar_relatorio():
    operacoes_df = pd.read_sql('SELECT * FROM Operacoes', conexao)
    operacoes_df['data'] = pd.to_datetime(operacoes_df['data'], format='%Y/%m/%d')
    operacoes_df.to_excel(r'C:\Users\Ariel\Documents\PYTHON HASHTAG\ENCONTROS AO VIVO\CONTROLE ESTOQUE\janela\Controle Estoque.xlsx', index=False)
    tkinter.messagebox.showinfo(title='Relatório Estoque', message='Relatório Atualizado')
    

window = Tk()
window.title('Controle de Estoque')

window.geometry("711x646")
window.configure(bg = "#ffffff")
canvas = Canvas(
    window,
    bg = "#ffffff",
    height = 646,
    width = 711,
    bd = 0,
    highlightthickness = 0,
    relief = "ridge")
canvas.place(x = 0, y = 0)

background_img = PhotoImage(file = f"background.png")
background = canvas.create_image(
    355.5, 323.0,
    image=background_img)

img0 = PhotoImage(file = f"img0.png")
b0 = Button(
    image = img0,
    borderwidth = 0,
    highlightthickness = 0,
    command = btn_clicked0,
    relief = "flat")

b0.place(
    x = 479, y = 195,
    width = 178,
    height = 38)

img1 = PhotoImage(file = f"img1.png")
b1 = Button(
    image = img1,
    borderwidth = 0,
    highlightthickness = 0,
    command = btn_clicked1,
    relief = "flat")

b1.place(
    x = 247, y = 197,
    width = 178,
    height = 36)

img2 = PhotoImage(file = f"img2.png")
b2 = Button(
    image = img2,
    borderwidth = 0,
    highlightthickness = 0,
    command = btn_clicked2,
    relief = "flat")

b2.place(
    x = 479, y = 123,
    width = 178,
    height = 35)

img3 = PhotoImage(file = f"img3.png")
b3 = Button(
    image = img3,
    borderwidth = 0,
    highlightthickness = 0,
    command = btn_clicked3,
    relief = "flat")

b3.place(
    x = 247, y = 125,
    width = 178,
    height = 34)

img4 = PhotoImage(file = f"Imagem1.png")
b4 = Button(
    image = img4,
    borderwidth = 0,
    highlightthickness = 0,
    command = gerar_relatorio,
    relief = "flat")

b4.place(
    x = 14, y = 590,
    width = 140,
    height = 25)


entry0_img = PhotoImage(file = f"img_textBox0.png")
entry0_bg = canvas.create_image(
    455.0, 560.0,
    image = entry0_img)

entry0 = Text(
    bd = 0,
    bg = "#ffffff",
    highlightthickness = 0)

entry0.place(
    x = 250, y = 502,
    width = 410,
    height = 114)

entry1_img = PhotoImage(file = f"img_textBox1.png")
entry1_bg = canvas.create_image(
    517.0, 294.5,
    image = entry1_img)

entry1 = Entry(
    bd = 0,
    bg = "#ffffff",
    highlightthickness = 0)

entry1.place(
    x = 377, y = 278,
    width = 280,
    height = 31)

entry2_img = PhotoImage(file = f"img_textBox2.png")
entry2_bg = canvas.create_image(
    517.0, 340.5,
    image = entry2_img)

entry2 = DateEntry(locale='pt_br',
    bd = 0,
    bg = "#ffffff",
    highlightthickness = 0)

entry2.place(
    x = 377, y = 324,
    width = 280,
    height = 35)

entry3_img = PhotoImage(file = f"img_textBox3.png")
entry3_bg = canvas.create_image(
    517.0, 388.5,
    image = entry3_img)

entry3 = Entry(
    bd = 0,
    bg = "#ffffff",
    highlightthickness = 0)

entry3.place(
    x = 377, y = 372,
    width = 280,
    height = 31)

entry4_img = PhotoImage(file = f"img_textBox4.png")
entry4_bg = canvas.create_image(
    517.0, 436.5,
    image = entry4_img)

entry4 = Entry(
    bd = 0,
    bg = "#ffffff",
    highlightthickness = 0)

entry4.place(
    x = 377, y = 420,
    width = 280,
    height = 31)

window.resizable(False, False)
window.mainloop()
